In [1]:
import sys
import numpy as np
import json
import transformers
from transformers import CamembertForMaskedLM
from transformers import CamembertTokenizer as token
from transformers import CamembertForSequenceClassification
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from transformers import CamembertConfig,CamembertTokenizer,CamembertModel
import tokenizers
import pandas as pd
sys.path.append('../')
import lib
import os
import torch

In [2]:
all_df=pd.DataFrame()
directory = os.path.join("../","data/data/")
for root,dirs,files in os.walk(directory):
    for file in files :
        if file.endswith(".json") and file.startswith("articles"):
            with open(root+file, 'r') as reader:
                data = json.load(reader)
            df = pd.DataFrame(columns=['id', 'title','text','Date','country','source', 'tweet'], 
                 data=data['articles'])
            all_df=pd.concat([all_df,df],axis=0,ignore_index=True)

In [3]:
all_df["text"]=all_df["text"].astype(str)

In [4]:
#884 rows with no text --> that's why we will create a column called tweet + text
print(len(all_df[all_df["text"].str.len()<1]))
#This means that there is no article so we will just add the tweet  to the text column (there is no link normally)
indexes=all_df[all_df["text"].str.len()<1].index
all_df.loc[indexes,"text"]=all_df.loc[indexes,"tweet"]

884


In [5]:
#verify it changed
len(all_df[all_df["text"].str.len()<1])

0

In [6]:
all_df["country"].unique()

array(['Maroc', 'Algeria', 'Guinee', 'Togo', 'Ile Maurice', 'Ivory Coast',
       'Cameroun', 'Niger', 'RDC', 'Burundi', 'Mali', 'Benin', 'Burkina',
       'Tunisia', 'Comores', 'Madagascar', 'Congo', 'Senegal', 'Gabon',
       'Centrafrique'], dtype=object)

In [7]:
id_=np.random.randint(all_df.shape[0])
print(id_)
all_df["text"].iloc[id_]

33565


"Le parc Murdoch est l’un des plus anciens espaces verts de Casablanca. Proche du Palais Royal et très prisé pour sa verdure et ses arbres centenaires, ce parc connaît une grande affluence depuis assez longtemps pour que l’on soit attristé par le sort qui lui est réservé de nos jours.\n\nLorsqu’il a été rénové et inauguré par Sa Majesté le Roi Mohammed VI, le 12 septembre 2006, le parc Murdoch, l’un des symboles de la capitale économique, est devenu, plus que jamais, un lieu idéal pour les sorties en famille. En effet, les nombreuses attractions pour enfants et les installations sportives qu’il accueille en son sein, attirent beaucoup de monde. Des visiteurs à la foi avides d’évasion et désireux de s’offrir une bulle d’oxygène dans une ville où la pollution sonore et aérienne bat son plein.\n\nMais malheureusement, c’est sans compter sur une frange de la population casablancaise qui envahit une fois la nuit tombée, ce petit coin de paradis citadin. Clochards de tous bords passent des h

In [28]:
# check errors for future tokenization 
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
tokens = tokenizer.tokenize("coronavirus Covid-19 +")
print("Tokens: {}".format(tokens))

#Probleme avec noms de famille africain Alassane Ouattara (president Cote D'ivoire)/Joseph Dion Ngute (President Cameroun)
tokens = tokenizer.tokenize("Donald Trump Obama  Nicolas Sarkozy ||| Alassane Ouattara Arnauld || Joseph Dion Ngute")
print("Tokens: {}".format(tokens))

#Should we replace presidents names by President ??

Tokens: ['▁cor', 'ona', 'virus', '▁Co', 'vid', '-19', '▁+']
Tokens: ['▁Donald', '▁Trump', '▁Obama', '▁Nicolas', '▁Sarkozy', '▁|', '|', '|', '▁Al', 'assan', 'e', '▁Ouattara', '▁Arn', 'au', 'ld', '▁|', '|', '▁Joseph', '▁Dion', '▁N', 'gu', 'te']


In [29]:
for column in ["tweet","text"]:
    all_df[column] = all_df[column].replace('\n', ' ',regex=True)
    all_df[column] = all_df[column].replace("  ", " ")
    all_df[column] = all_df[column].replace('[^\w\s]','')
    all_df[column] = all_df[column].replace('Covid-19','coronavirus')
    all_df[column] = all_df[column].replace('covid-19','coronavirus')
    all_df[column] = all_df[column].replace('covid 19','coronavirus')
    all_df[column] = all_df[column].replace('Covid 19','coronavirus')


In [30]:
id_=np.random.randint(all_df.shape[0])
print(id_)
all_df["text"].iloc[id_]

19856


'Société  Coronavirus  Les entreprises privées peuvent mettre en place leurs propres organisations pour rester opérationnelles. Contrairement au secteur public, le secteur privé a le choix de fermer totalement leurs portes ou de poursuivre leurs activités, tout en respectant certaines conditions imposées par le gouvernement concernant la protection sanitaire de leurs employés. En proclamant ces mesures, dans le cadre du confinement des deux principales régions de l’île, Analamanga et Antsinanana, pour les protéger contre la propagation du coronavirus, le président de la République, Andry Rajoelina veut surtout préserver l’économie nationale.  En effet, analystes et économistes s’accordent à dire que cette crise sanitaire et les mesures connexes prises au niveau de chaque pays touché auront d’importants impacts sur l’économie de chaque pays, et au niveau mondial. Les pays en voie de développement comme Madagascar encourent notamment d’importants dégâts économiques si aucune mesure n’est

In [31]:
all_df.to_csv(directory+"/Data_cleaned.csv",index=False)

In [11]:
# Store the model we want to use
MODEL_NAME = "camembert-base"

# 2 POSSIBILITIES TO LOAD CAMEMBERT

#1st
config = CamembertConfig.from_pretrained(MODEL_NAME)    # Download configuration from S3 and cache.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_config(config) 


#2nd
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
camembert = CamembertModel.from_pretrained("camembert-base")

camembert.eval() 

CamembertModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, in

In [12]:
# Tokens comes from a process that splits the input into sub-entities with interesting linguistic properties. 
tokens = tokenizer.tokenize("ceci est un exemple !")
print("Tokens: {}".format(tokens))

# This is not sufficient for the model, as it requires integers as input, 
# not a problem, let's convert tokens to ids.
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Tokens id: {}".format(tokens_ids))

# Add the required special tokens
tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)

# We need to convert to a Deep Learning framework specific format, let's use PyTorch for now.
tokens_pt = torch.tensor([tokens_ids])
print("Tokens PyTorch: {}".format(tokens_pt))

# Now we're ready to go through BERT with out input
outputs, pooled = model(tokens_pt)
print("Token wise output: {}, Pooled output: {}".format(outputs.shape, pooled.shape))

Tokens: ['▁ceci', '▁est', '▁un', '▁exemple', '▁!']
Tokens id: [2384, 30, 23, 411, 83]
Tokens PyTorch: tensor([[   5, 2384,   30,   23,  411,   83,    6]])
Token wise output: torch.Size([1, 7, 768]), Pooled output: torch.Size([1, 768])


In [13]:
tokens = tokenizer.tokenize('Covid-19')

In [14]:
directory

'../data/data/'